## Project of Machine Learning

Purpose : Find the best roommate with a data of Wellbeing and lifestyle

for this project, we use the dataset from Kaggle : https://www.kaggle.com/c/wellbeing-and-lifestyle/data

we must define whether we use classification or regression


In [23]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import os


In [24]:
file_path = os.path.join(os.getcwd(), "Wellbeing_and_lifestyle_data.csv")

try:
    df = pd.read_csv(file_path)
except FileNotFoundError:
    print("File not found")

print(df.describe())
print(df.head())



       FRUITS_VEGGIES  DAILY_STRESS  PLACES_VISITED   CORE_CIRCLE  \
count    15971.000000  15971.000000    15971.000000  15971.000000   
mean         2.922672      2.791685        5.233235      5.508296   
std          1.442739      1.367801        3.311847      2.840287   
min          0.000000      0.000000        0.000000      0.000000   
25%          2.000000      2.000000        2.000000      3.000000   
50%          3.000000      3.000000        5.000000      5.000000   
75%          4.000000      4.000000        8.000000      8.000000   
max          5.000000      5.000000       10.000000     10.000000   

       SUPPORTING_OTHERS  SOCIAL_NETWORK   ACHIEVEMENT      DONATION  \
count       15971.000000    15971.000000  15971.000000  15971.000000   
mean            5.616179        6.474047      4.000689      2.715171   
std             3.241937        3.086643      2.755912      1.851556   
min             0.000000        0.000000      0.000000      0.000000   
25%             3.

# Data exploration

## Types of Data
dtypes: float64(1), int64(19), object(4)

Object : Age, Gender, Timestamp, DAILY_STRESS

Float64 : Work_Life_Balance_Score

We must transform the Object into numerical values

For DAILY_STRESS => we had one data (1/1/0) who isn't INT on around 15K samples), result: we delete the row




In [25]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15971 entries, 0 to 15970
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Timestamp                15971 non-null  object 
 1   FRUITS_VEGGIES           15971 non-null  int64  
 2   DAILY_STRESS             15971 non-null  int64  
 3   PLACES_VISITED           15971 non-null  int64  
 4   CORE_CIRCLE              15971 non-null  int64  
 5   SUPPORTING_OTHERS        15971 non-null  int64  
 6   SOCIAL_NETWORK           15971 non-null  int64  
 7   ACHIEVEMENT              15971 non-null  int64  
 8   DONATION                 15971 non-null  int64  
 9   BMI_RANGE                15971 non-null  int64  
 10  TODO_COMPLETED           15971 non-null  int64  
 11  FLOW                     15971 non-null  int64  
 12  DAILY_STEPS              15971 non-null  int64  
 13  LIVE_VISION              15971 non-null  int64  
 14  SLEEP_HOURS           

In [26]:
#GENDER
# onyl 2 values (Female and Male), so we translate this into 0 and 1
print(df["GENDER"].unique())

df["GENDER"] = df["GENDER"].map({"Female": 0, "Male": 1})

#Age
#We have 4 categories of Age :
#[less than 20, 21 to 35, 36 to 50, 51 or more]
# when it's possible, we take the half of age groupes
# for <20, we choose value : 16 and for 51 or more, we choose value : 70

# Convert AGE to numeric representation
age_mapping = {
    'Less than 20': 16,
    '21 to 35': 28,
    '36 to 50': 43,
    '51 or more': 70
}
df['AGE'] = df['AGE'].map(age_mapping)


['Female' 'Male']


## Training of the AI



In [27]:

scaler = MinMaxScaler()
normalized_df = scaler.fit_transform(df)
X_train, X_test, y_train, y_test = train_test_split(normalized_df, test_size=0.2, random_state=42)



ValueError: could not convert string to float: '7/7/15'